In [1]:
def load_data():
    # load data
    import os
    import numpy as np

    data_dir = os.path.join(os.getcwd(), "data")
    train_path = os.path.join(data_dir, 'train.txt')
    valid_path = os.path.join(data_dir, 'valid.txt')
    test_path  = os.path.join(data_dir, 'test.txt')

    with open(train_path, 'r', encoding="utf-8") as f:
        train_raw = [l.strip() for l in f.readlines()]
        train = list()
        start_of_sentence = 0
        for i in range(len(train_raw)):
            if train_raw[i] == '': # end of sentence
                sent = list()
                tags = list()
                for l in train_raw[start_of_sentence:i]:
                    l = l.split(' ')
                    sent.append(l[0])
                    tags.append(l[-1])
                train.append((sent, tags))
                start_of_sentence = i+1

    with open(valid_path, 'r', encoding="utf-8") as f:
        valid_raw = [l.strip() for l in f.readlines()]
        valid = list()
        start_of_sentence = 0
        for i in range(len(valid_raw)):
            if valid_raw[i] == '': # end of sentence
                sent = list()
                tags = list()
                for l in valid_raw[start_of_sentence:i]:
                    l = l.split(' ')
                    sent.append(l[0])
                    tags.append(l[-1])
                valid.append((sent, tags))
                start_of_sentence = i+1
    with open(test_path, 'r', encoding="utf-8") as f:
        test_raw = [l.strip() for l in f.readlines()]
        test = list()
        start_of_sentence = 0
        for i in range(len(test_raw)):
            if test_raw[i] == '': # end of sentence
                sent = list()
                tags = list()
                for l in test_raw[start_of_sentence:i]:
                    l = l.split(' ')
                    sent.append(l[0])
                    tags.append(l[-1])
                test.append((sent, tags))
                start_of_sentence = i+1
    
    return train, valid, test

In [2]:
train, valid, test = load_data()
# print(train)
# print(valid)
# print(test)

In [31]:
def export_to_file(export_file_path, data):
  tag_to_idx = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}
  with open(export_file_path, "w") as f:
      for tokens, tags in data:
          if len(tokens) > 0:
              f.write(
                  str(len(tokens))
                  + "\t"
                  + "\t".join(tokens)
                  + "\t"
                  + "\t".join([str(tag_to_idx[tag]) for tag in tags])
                  + "\n"
              )


# os.mkdir("data")
export_to_file("./data/trans_train.txt", train)
export_to_file("./data/trans_valid.txt", valid)
export_to_file("./data/trans_test.txt", test)

## Install the open source datasets library from HuggingFace

In [3]:
!pip3 install datasets
!wget https://raw.githubusercontent.com/sighsmile/conlleval/master/conlleval.py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2022-11-21 15:16:27--  https://raw.githubusercontent.com/sighsmile/conlleval/master/conlleval.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7502 (7.3K) [text/plain]
Saving to: ‘conlleval.py.1’

conlleval.py.1      100%[===================>]   7.33K  --.-KB/s    in 0s      

2022-11-21 15:16:27 (57.6 MB/s) - ‘conlleval.py.1’ saved [7502/7502]



In [16]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from collections import Counter
from conlleval import evaluate

We will be using the transformer implementation from this fantastic
[example](https://keras.io/examples/nlp/text_classification_with_transformer/).

Let's start by defining a `TransformerBlock` layer:

In [5]:

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.ffn = keras.Sequential(
            [
                keras.layers.Dense(ff_dim, activation="relu"),
                keras.layers.Dense(embed_dim),
            ]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


Next, let's define a `TokenAndPositionEmbedding` layer:

In [6]:

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, inputs):
        maxlen = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        position_embeddings = self.pos_emb(positions)
        token_embeddings = self.token_emb(inputs)
        return token_embeddings + position_embeddings


## Build the NER model class as a `keras.Model` subclass

In [7]:

class NERModel(keras.Model):
    def __init__(
        self, num_tags, vocab_size, maxlen=128, embed_dim=32, num_heads=2, ff_dim=32
    ):
        super(NERModel, self).__init__()
        self.embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
        self.transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
        self.dropout1 = layers.Dropout(0.1)
        self.ff = layers.Dense(ff_dim, activation="relu")
        self.dropout2 = layers.Dropout(0.1)
        self.ff_final = layers.Dense(num_tags, activation="softmax")

    def call(self, inputs, training=False):
        x = self.embedding_layer(inputs)
        x = self.transformer_block(x)
        x = self.dropout1(x, training=training)
        x = self.ff(x)
        x = self.dropout2(x, training=training)
        x = self.ff_final(x)
        return x


## Make the NER label lookup table

Note that we start our label numbering from 1 since 0 will be reserved for padding. We
have a total of 10 labels: 9 from the NER dataset and one for padding.

In [17]:

def make_tag_lookup_table():
    iob_labels = ["B", "I"]
    ner_labels = ["PER", "ORG", "LOC", "MISC"]
    all_labels = [(label1, label2) for label2 in ner_labels for label1 in iob_labels]
    all_labels = ["-".join([a, b]) for a, b in all_labels]
    all_labels = ["[PAD]", "O"] + all_labels
    return dict(zip(range(0, len(all_labels) + 1), all_labels))


mapping = make_tag_lookup_table()
print(mapping)

tag_to_idx = {mapping[idx]: idx for idx in mapping}
print(tag_to_idx)

{0: '[PAD]', 1: 'O', 2: 'B-PER', 3: 'I-PER', 4: 'B-ORG', 5: 'I-ORG', 6: 'B-LOC', 7: 'I-LOC', 8: 'B-MISC', 9: 'I-MISC'}
{'[PAD]': 0, 'O': 1, 'B-PER': 2, 'I-PER': 3, 'B-ORG': 4, 'I-ORG': 5, 'B-LOC': 6, 'I-LOC': 7, 'B-MISC': 8, 'I-MISC': 9}


Get a list of all tokens in the training dataset. This will be used to create the
vocabulary.

In [18]:
all_tokens = []
for record in train:
  for token in record[0]:
    all_tokens.append(token)
    
all_tokens_array = np.array(list(map(str.lower, all_tokens)))

counter = Counter(all_tokens_array)
print(len(counter))

num_tags = len(mapping)
vocab_size = 20000

# We only take (vocab_size - 2) most commons words from the training data since
# the `StringLookup` class uses 2 additional tokens - one denoting an unknown
# token and another one denoting a masking token
vocabulary = [token for token, count in counter.most_common(vocab_size - 2)]

# The StringLook class will convert tokens to token IDs
lookup_layer = keras.layers.StringLookup(
    vocabulary=vocabulary
)

21010


Create 2 new `Dataset` objects from the training and validation data

In [11]:
# train_data = tf.data.TextLineDataset("./data/conll_train.txt")
# val_data = tf.data.TextLineDataset("./data/conll_val.txt")

In [26]:
data_dir = os.path.join(os.getcwd(), "data")
tans_train_path = os.path.join(data_dir, 'trans_train.txt')
tans_valid_path = os.path.join(data_dir, 'trans_valid.txt')
tans_test_path  = os.path.join(data_dir, 'trans_test.txt')

train_data = tf.data.TextLineDataset(tans_train_path)
val_data = tf.data.TextLineDataset(tans_valid_path)
test_data = tf.data.TextLineDataset(tans_test_path)

# train_data = tf.data.Dataset.from_tensor_slices(train)
# valid_data = tf.data.Dataset.from_tensor_slices(valid)
# test_data = tf.data.Dataset.from_tensor_slices(test)

Print out one line to make sure it looks good. The first record in the line is the number of tokens.
After that we will have all the tokens followed by all the ner tags.

In [24]:
print(list(train_data.take(2).as_numpy_iterator()))

[b'1\t-DOCSTART-\t0', b'9\tEU\trejects\tGerman\tcall\tto\tboycott\tBritish\tlamb\t.\t3\t0\t7\t0\t0\t0\t7\t0\t0']


We will be using the following map function to transform the data in the dataset:

In [27]:

def map_record_to_training_data(record):
    record = tf.strings.split(record, sep="\t")
    length = tf.strings.to_number(record[0], out_type=tf.int32)
    tokens = record[1 : length + 1]
    tags = record[length + 1 :]
    tags = tf.strings.to_number(tags, out_type=tf.int64)
    tags += 1
    return tokens, tags


def lowercase_and_convert_to_ids(tokens):
    tokens = tf.strings.lower(tokens)
    return lookup_layer(tokens)


# We use `padded_batch` here because each record in the dataset has a
# different length.
batch_size = 32
train_dataset = (
    train_data.map(map_record_to_training_data)
    .map(lambda x, y: (lowercase_and_convert_to_ids(x), y))
    .padded_batch(batch_size)
)
val_dataset = (
    val_data.map(map_record_to_training_data)
    .map(lambda x, y: (lowercase_and_convert_to_ids(x), y))
    .padded_batch(batch_size)
)

ner_model = NERModel(num_tags, vocab_size, embed_dim=32, num_heads=4, ff_dim=64)

We will be using a custom loss function that will ignore the loss from padded tokens.

In [28]:

class CustomNonPaddingTokenLoss(keras.losses.Loss):
    def __init__(self, name="custom_ner_loss"):
        super().__init__(name=name)

    def call(self, y_true, y_pred):
        loss_fn = keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, reduction=keras.losses.Reduction.NONE
        )
        loss = loss_fn(y_true, y_pred)
        mask = tf.cast((y_true > 0), dtype=tf.float32)
        loss = loss * mask
        return tf.reduce_sum(loss) / tf.reduce_sum(mask)


loss = CustomNonPaddingTokenLoss()

## Compile and fit the model

In [29]:
ner_model.compile(optimizer="adam", loss=loss)
ner_model.fit(train_dataset, epochs=10)


def tokenize_and_convert_to_ids(text):
    tokens = text.split()
    return lowercase_and_convert_to_ids(tokens)


# Sample inference using the trained model
sample_input = tokenize_and_convert_to_ids(
    "eu rejects german call to boycott british lamb"
)
sample_input = tf.reshape(sample_input, shape=[1, -1])
print(sample_input)

output = ner_model.predict(sample_input)
prediction = np.argmax(output, axis=-1)[0]
prediction = [mapping[i] for i in prediction]

# eu -> B-ORG, german -> B-MISC, british -> B-MISC
print(prediction)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


469/469 [==============================] - 24s 47ms/step - loss: 0.6005
Epoch 2/10
469/469 [==============================] - 21s 44ms/step - loss: 0.2515
Epoch 3/10
469/469 [==============================] - 21s 44ms/step - loss: 0.1533
Epoch 4/10
469/469 [==============================] - 20s 43ms/step - loss: 0.1186
Epoch 5/10
469/469 [==============================] - 21s 44ms/step - loss: 0.0964
Epoch 6/10
469/469 [==============================] - 22s 47ms/step - loss: 0.0791
Epoch 7/10
469/469 [==============================] - 20s 44ms/step - loss: 0.0690
Epoch 8/10
469/469 [==============================] - 20s 43ms/step - loss: 0.0599
Epoch 9/10
469/469 [==============================] - 23s 48ms/step - loss: 0.0530
Epoch 10/10
469/469 [==============================] - 21s 44ms/step - loss: 0.0448
tf.Tensor([[  989 10951   205   629     6  3939   216  5774]], shape=(1, 8), dtype=int64)
1/1 [==============================] - 0s 281ms/step
['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O

## Metrics calculation

Here is a function to calculate the metrics. The function calculates F1 score for the
overall NER dataset as well as individual scores for each NER tag.

In [32]:

def calculate_metrics(dataset):
    all_true_tag_ids, all_predicted_tag_ids = [], []

    for x, y in dataset:
        output = ner_model.predict(x)
        predictions = np.argmax(output, axis=-1)
        predictions = np.reshape(predictions, [-1])

        true_tag_ids = np.reshape(y, [-1])

        mask = (true_tag_ids > 0) & (predictions > 0)
        true_tag_ids = true_tag_ids[mask]
        predicted_tag_ids = predictions[mask]

        all_true_tag_ids.append(true_tag_ids)
        all_predicted_tag_ids.append(predicted_tag_ids)

    all_true_tag_ids = np.concatenate(all_true_tag_ids)
    all_predicted_tag_ids = np.concatenate(all_predicted_tag_ids)

    predicted_tags = [mapping[tag] for tag in all_predicted_tag_ids]
    real_tags = [mapping[tag] for tag in all_true_tag_ids]

    evaluate(real_tags, predicted_tags)


calculate_metrics(val_dataset)

1/1 [==============================] - 0s 23ms/step
processed 51578 tokens with 5942 phrases; found: 5588 phrases; correct: 4142.
accuracy:  65.62%; (non-O)
accuracy:  93.60%; precision:  74.12%; recall:  69.71%; FB1:  71.85
              LOC: precision:  83.81%; recall:  81.16%; FB1:  82.47  1779
             MISC: precision:  76.76%; recall:  68.76%; FB1:  72.54  826
              ORG: precision:  68.19%; recall:  61.07%; FB1:  64.44  1201
              PER: precision:  67.23%; recall:  65.04%; FB1:  66.11  1782


## Conclusions

In this exercise, we created a simple transformer based named entity recognition model.
We trained it on the CoNLL 2003 shared task data and got an overall F1 score of around 70%.
State of the art NER models fine-tuned on pretrained models such as BERT or ELECTRA can easily
get much higher F1 score -between 90-95% on this dataset owing to the inherent knowledge
of words as part of the pretraining process and the usage of subword tokenization.

You can use the trained model hosted on [Hugging Face Hub](https://huggingface.co/keras-io/ner-with-transformers) and try the demo on [Hugging Face Spaces](https://huggingface.co/spaces/keras-io/ner_with_transformers).